In [1]:
from utils import *

import folium
import geopandas as gpd

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.metrics import mean_squared_log_error, mean_absolute_error
from sklearn.model_selection import train_test_split, cross_val_score, KFold
from sklearn.tree import DecisionTreeRegressor
from sklearn.pipeline import make_pipeline
from sklearn.ensemble import RandomForestRegressor
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler

from xgboost import XGBRegressor

## Data description

In [2]:
macro_df = pd.read_csv('data/macro.csv', parse_dates=['timestamp'])
train_df = pd.read_csv('data/train.csv', index_col='id', parse_dates=['timestamp'])
test_df = pd.read_csv('data/test.csv', index_col='id', parse_dates=['timestamp'])
tverskoe_issue_fix(train_df)
tverskoe_issue_fix(test_df)

Fix:  550
Fix:  149


## 1. Data preprocessing
## I part (encoding and correcting mistakes)

### Macro dataset

In [3]:
macro_df['child_on_acc_pre_school'] = macro_df['child_on_acc_pre_school'].str.replace('#!', 'nan')
for column in macro_df.select_dtypes('object').columns:
    macro_df[column] = macro_df[column].str.replace(',', '.')
    macro_df[column] = macro_df[column].astype(float)

if not len(macro_df.select_dtypes('object').columns):
    print('OK')

OK


### Train dataset

In [4]:
train_df = encode(train_df)

### Test dataset

In [5]:
test_df = encode(test_df)

## II part (Filling missing values)

XGBRegressor model handles `np.NaN` values itself

## 2. Encoding `sub_area` feature

In [6]:
train_df['is_train'] = 1
test_df['is_train'] = 0

coords_df = pd.read_csv('data/coords.csv', index_col='id')
all_df = pd.concat([train_df, test_df])

all_df['latitude'] = coords_df['latitude']
all_df['longitude'] = coords_df['longitude']

## 3. Removing outliers

In [7]:
all_df = remove_outliers(all_df)

In [8]:
train_df = all_df[all_df['is_train'] == 1].drop(['is_train'], axis=1)
train_df = remove_fake_prices(train_df)
test_df = all_df[all_df['is_train'] == 0].drop(['is_train', 'price_doc'], axis=1)

## 4. Modeling

### `product_type == 'OwnerOccupier'`

In [9]:
train_df_1 = train_df[train_df['product_type'] == 1]
test_df_1 = test_df[test_df['product_type'] == 1]

X_1 = train_df_1.drop(['sub_area', 'price_doc'], axis=1).copy()
y_1 = np.log1p(train_df_1['price_doc'])
X_test_1 = test_df_1.drop(['sub_area'], axis=1).copy()

In [10]:
X_train_1, X_val_1, y_train_1, y_val_1 = train_test_split(X_1, y_1, test_size=0.2, random_state=42)
X_train_1.shape, X_val_1.shape

((8813, 293), (2204, 293))

In [11]:
params = {'objective': 'reg:squarederror',
          'n_estimators': 10000,
          'tree_method': 'gpu_hist',
          'booster': 'gbtree',
          'base_score': 7,
          'learning_rate': 0.05,
          'max_depth': 4,
          'min_child_weight': 7,
          'subsample': 1,
          'colsample_bytree': 0.9,
          'reg_lambda': 5,
          'reg_alpha': 1,
          'eval_metric': 'rmse',
          'seed': 42,
          'nthread': -1
          }

model_1 = XGBRegressor(**params)

model_1.fit(X_train_1, y_train_1, eval_set=[(X_train_1, y_train_1), (X_val_1, y_val_1)], early_stopping_rounds=50)

[0]	validation_0-rmse:8.14545	validation_1-rmse:8.16014
[1]	validation_0-rmse:7.73947	validation_1-rmse:7.75311
[2]	validation_0-rmse:7.35379	validation_1-rmse:7.36645
[3]	validation_0-rmse:6.98736	validation_1-rmse:6.99924
[4]	validation_0-rmse:6.63926	validation_1-rmse:6.65074
[5]	validation_0-rmse:6.30859	validation_1-rmse:6.31916
[6]	validation_0-rmse:5.99441	validation_1-rmse:6.00429
[7]	validation_0-rmse:5.69597	validation_1-rmse:5.70550
[8]	validation_0-rmse:5.41241	validation_1-rmse:5.42143
[9]	validation_0-rmse:5.14306	validation_1-rmse:5.15156
[10]	validation_0-rmse:4.88715	validation_1-rmse:4.89496
[11]	validation_0-rmse:4.64403	validation_1-rmse:4.65102
[12]	validation_0-rmse:4.41308	validation_1-rmse:4.41947
[13]	validation_0-rmse:4.19367	validation_1-rmse:4.19965
[14]	validation_0-rmse:3.98524	validation_1-rmse:3.99055
[15]	validation_0-rmse:3.78724	validation_1-rmse:3.79218
[16]	validation_0-rmse:3.59910	validation_1-rmse:3.60338
[17]	validation_0-rmse:3.42040	validation

XGBRegressor(base_score=7, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=0.9, eval_metric='rmse',
             gamma=0, gpu_id=0, importance_type='gain',
             interaction_constraints='', learning_rate=0.05, max_delta_step=0,
             max_depth=4, min_child_weight=7, missing=nan,
             monotone_constraints='()', n_estimators=10000, n_jobs=8,
             nthread=-1, num_parallel_tree=1, random_state=42, reg_alpha=1,
             reg_lambda=5, scale_pos_weight=1, seed=42, subsample=1,
             tree_method='gpu_hist', validate_parameters=1, verbosity=None)

In [12]:
y_pred_train = np.abs(model_1.predict(X_train_1))
y_pred_val = np.abs(model_1.predict(X_val_1))
print('___________________________________')
print('      RMSLE\n')
print('      TRAIN_1:', np.sqrt(mean_squared_log_error(np.expm1(y_train_1), np.expm1(y_pred_train))))
print(' VALIDATION_1:', np.sqrt(mean_squared_log_error(np.expm1(y_val_1), np.expm1(y_pred_val))))
print('___________________________________')
print('      MAE\n')
print('      TRAIN_1:', mean_absolute_error(np.expm1(y_train_1), np.expm1(y_pred_train)))
print(' VALIDATION_1:', mean_absolute_error(np.expm1(y_val_1), np.expm1(y_pred_val)))

___________________________________
      RMSLE

      TRAIN_1: 0.058131259010291834
 VALIDATION_1: 0.10153342188740434
___________________________________
      MAE

      TRAIN_1: 274354.9766821742
 VALIDATION_1: 431269.2297527225


### `product_type == 'Investment'`

In [13]:
train_df_0 = train_df[train_df['product_type'] == 0]
test_df_0 = test_df[test_df['product_type'] == 0]

X_0 = train_df_0.drop(['sub_area', 'price_doc'], axis=1).copy()
y_0 = np.log1p(train_df_0['price_doc'])
X_test_0 = test_df_0.drop(['sub_area'], axis=1).copy()

In [14]:
X_train_0, X_val_0, y_train_0, y_val_0 = train_test_split(X_0, y_0, test_size=0.2, random_state=42)
X_train_0.shape, X_val_0.shape

((13916, 293), (3480, 293))

In [15]:
params = {'objective': 'reg:squarederror',
          'n_estimators': 10000,
          'tree_method': 'gpu_hist',
          'booster': 'gbtree',
          'base_score': 7,
          'learning_rate': 0.05,
          'max_depth': 4,
          'min_child_weight': 7,
          'subsample': 1,
          'colsample_bytree': 0.9,
          'reg_lambda': 5,
          'reg_alpha': 1,
          'eval_metric': 'rmse',
          'seed': 42,
          'nthread': -1
          }

model_0 = XGBRegressor(**params)

model_0.fit(X_train_0, y_train_0, eval_set=[(X_train_0, y_train_0), (X_val_0, y_val_0)], early_stopping_rounds=50)

[0]	validation_0-rmse:8.37687	validation_1-rmse:8.37489
[1]	validation_0-rmse:7.95922	validation_1-rmse:7.95715
[2]	validation_0-rmse:7.56252	validation_1-rmse:7.56037
[3]	validation_0-rmse:7.18565	validation_1-rmse:7.18344
[4]	validation_0-rmse:6.82765	validation_1-rmse:6.82541
[5]	validation_0-rmse:6.48762	validation_1-rmse:6.48532
[6]	validation_0-rmse:6.16458	validation_1-rmse:6.16226
[7]	validation_0-rmse:5.85775	validation_1-rmse:5.85541
[8]	validation_0-rmse:5.56629	validation_1-rmse:5.56392
[9]	validation_0-rmse:5.28945	validation_1-rmse:5.28706
[10]	validation_0-rmse:5.02649	validation_1-rmse:5.02402
[11]	validation_0-rmse:4.77673	validation_1-rmse:4.77407
[12]	validation_0-rmse:4.53950	validation_1-rmse:4.53680
[13]	validation_0-rmse:4.31417	validation_1-rmse:4.31153
[14]	validation_0-rmse:4.10015	validation_1-rmse:4.09753
[15]	validation_0-rmse:3.89689	validation_1-rmse:3.89423
[16]	validation_0-rmse:3.70383	validation_1-rmse:3.70120
[17]	validation_0-rmse:3.52049	validation

XGBRegressor(base_score=7, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=0.9, eval_metric='rmse',
             gamma=0, gpu_id=0, importance_type='gain',
             interaction_constraints='', learning_rate=0.05, max_delta_step=0,
             max_depth=4, min_child_weight=7, missing=nan,
             monotone_constraints='()', n_estimators=10000, n_jobs=8,
             nthread=-1, num_parallel_tree=1, random_state=42, reg_alpha=1,
             reg_lambda=5, scale_pos_weight=1, seed=42, subsample=1,
             tree_method='gpu_hist', validate_parameters=1, verbosity=None)

In [16]:
y_pred_train = np.abs(model_0.predict(X_train_0))
y_pred_val = np.abs(model_0.predict(X_val_0))
print('___________________________________')
print('      RMSLE\n')
print('      TRAIN_1:', np.sqrt(mean_squared_log_error(np.expm1(y_train_0), np.expm1(y_pred_train))))
print(' VALIDATION_1:', np.sqrt(mean_squared_log_error(np.expm1(y_val_0), np.expm1(y_pred_val))))
print('___________________________________')
print('      MAE\n')
print('      TRAIN_1:', mean_absolute_error(np.expm1(y_train_0), np.expm1(y_pred_train)))
print(' VALIDATION_1:', mean_absolute_error(np.expm1(y_val_0), np.expm1(y_pred_val)))

___________________________________
      RMSLE

      TRAIN_1: 0.2297278099989123
 VALIDATION_1: 0.28667372484083364
___________________________________
      MAE

      TRAIN_1: 953739.1728945102
 VALIDATION_1: 1220016.825


In [17]:
test_df_nan = test_df[test_df['product_type'].isna()]
X_test_nan = test_df_nan.drop(['sub_area'], axis=1).copy()

In [18]:
submission = pd.read_csv('data/sample_submission.csv', index_col='id')
pred_0 = np.expm1(model_0.predict(X_test_0))
pred_1 = np.expm1(model_1.predict(X_test_1))
pred_nan = np.expm1(model_1.predict(X_test_nan))

if len(pred_0[pred_0 < 0]) or len(pred_1[pred_1 < 0]) or len(pred_nan[pred_nan < 0]):
    print('WARNING: NEGATIVE PREDICTIONS')

In [36]:
submission.loc[X_test_0.index, 'price_doc'] = 0.9*pred_0
submission.loc[X_test_1.index, 'price_doc'] = 0.93*pred_1
submission.loc[X_test_nan.index, 'price_doc'] = pred_nan
submission.to_csv('submits/submission.csv', index='id')

In [37]:
!kaggle competitions submit -c sberbank-russian-housing-market -f "submits/submission.csv" -m "XGBRegressor tuned"

100%|████████████████████████████████████████| 121k/121k [00:03<00:00, 40.2kB/s]
Successfully submitted to Sberbank Russian Housing Market

In [21]:
# !kaggle competitions submissions -c sberbank-russian-housing-market